In [49]:
"""
날짜 : 2020/08/20
이름 : 유효진
내용 : LSTM imdb 텍스트 분석 실습하기
"""
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, LSTM, Dropout

from tensorflow.keras.layers import GlobalMaxPool1D

In [50]:
#데이터셋 로드하기
(imdb_train_data, imdb_train_label), (imdb_test_data, imdb_test_label) = imdb.load_data(num_words=1000)

In [51]:
#데이터 확인하기
print(imdb_train_data.shape, imdb_train_label.shape)
print(imdb_test_data.shape, imdb_test_label.shape)

# 영문을 숫자로 변환한 형태로 출력
#imdb_train_data[0]

# 긍정 1, 부정 0
#imdb_train_label

#시퀀스데이터 영어 문장으로 변환
imdb_get_word_index = {}

for word, value in imdb.get_word_index().items():
    #print(word, '--->', value)
    imdb_get_word_index[value] = word
    
#imdb_get_word_index[1]

for w in imdb_train_data[0]:
    print(imdb_get_word_index[w], end=' ')
    
    '''
    데이터셋 로드 시 단어 수를 1000개로 제한해서 온전한 문장이 아님
    가장 많이 언급된 단어들만 수집한 것
    '''

(25000,) (25000,)
(25000,) (25000,)
the as you with out themselves powerful and and their becomes and had and of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every and and movie except her was several of enough more with is now and film as you of and and unfortunately of you than him that with out themselves her get for was and of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of and and with heart had and they of here that with her serious to have does when from why what have and they is you that isn't one will very to as itself with other and in of seen over and for anyone of and br and to whether from than out themselves history he name half some br of and and was two most of mean for 1 any an and she he should is thought and but of script you not while history he heart to real at and but when from one bit then have two of script their with 

In [52]:
#데이터 전처리(데이터를 동일한 길이로 맞추기)

print('패딩 전 :', len(imdb_train_data[0]))
print('패딩 전 :', len(imdb_train_data[1]))

pad_imdb_train_data = pad_sequences(imdb_train_data, maxlen=500, padding='pre')
pad_imdb_test_data = pad_sequences(imdb_test_data, maxlen=500, padding='pre')

print('패딩 후 :', len(pad_imdb_train_data[0]))
print('패딩 후 :', len(pad_imdb_train_data[1]))

패딩 전 : 218
패딩 전 : 189
패딩 후 : 500
패딩 후 : 500


In [70]:
#모델 구성하기
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=128))
model.add(LSTM(128, return_sequences=True)) #순환층
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) #출력층

In [ ]:
"""
https://wikidocs.net/24586

검증 데이터의 손실(loss)이 증가하면,
과적합 징후이므로 검증 데이터 손실이 4회 증가하면 학습을 중단하는 조기 종료(EarlyStopping)를 사용합니다.
또한, ModelCheckpoint를 사용하여 검증 데이터의 정확도가 이전보다 좋아질 경우에만 모델을 저장하도록 합니다.

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
"""

In [71]:
#모델 설정하기
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])

In [ ]:
#모델 학습하기
model.fit(pad_imdb_train_data,
          imdb_test_label,
          batch_size=32,
          epochs=10,
          validation_split=0.2)

Epoch 1/10
211/625 [=========>....................] - ETA: 1:27 - loss: 0.6936 - acc: 0.5092

In [64]:
#모델 평가하기
model.evaluate(pad_imdb_test_data, imdb_test_label)

782/782 [==============================] - 16s 21ms/step - loss: 0.8971 - acc: 0.4871


[0.8971473574638367, 0.48712000250816345]

In [ ]:
#예측하기
sample_text = ['i love this movie', 'it is waste of time']
result = model.predict(sample_text)
result